# The Paradox of Your Mother's Education

This notebook explores fertility trends in the US population. The focus of this piece is on the fertility differences between education segments.

- [Population Trends](#population-trends)
- [Fertility Trends by Education Segments](#fertility-trends-by-education-segments)
- [Advancing the Statistic](#advancing-the-statistic)
- [The Paradox](#the-paradox)

In [1]:
import pandas as pd

df = pd.read_csv("data/clean/biannual_census_2012-2020.csv")
df.head()

,Year,Education,Age Group,Marital Status,Children Ever Born per Woman,Total Women,Childlessness
0,2012,Associate's degree,15 - 19,All Marital Statuses,0.000,63000.0,1.000
1,2012,Bachelor's degree,15 - 19,All Marital Statuses,0.047,28000.0,0.953
2,2012,Graduate or professional degree,15 - 19,All Marital Statuses,0.524,7000.0,0.476
3,2012,High school graduate,15 - 19,All Marital Statuses,0.166,1481000.0,0.860
4,2012,Not a high school graduate,15 - 19,All Marital Statuses,0.045,7509000.0,0.967


In [2]:
df["Total Children"] = df["Children Ever Born per Woman"] * df["Total Women"]
df.head()

,Year,Education,Age Group,Marital Status,Children Ever Born per Woman,Total Women,Childlessness,Total Children
0,2012,Associate's degree,15 - 19,All Marital Statuses,0.000,63000.0,1.000,0.0
1,2012,Bachelor's degree,15 - 19,All Marital Statuses,0.047,28000.0,0.953,1316.0
2,2012,Graduate or professional degree,15 - 19,All Marital Statuses,0.524,7000.0,0.476,3668.0
3,2012,High school graduate,15 - 19,All Marital Statuses,0.166,1481000.0,0.860,245846.0
4,2012,Not a high school graduate,15 - 19,All Marital Statuses,0.045,7509000.0,0.967,337905.0


In [3]:
# Calculate the number of children born to each age group for each year

df = df.filter(
    df[~df["Age Group"].isin(["15 - 19", "15 - 44", "15 - 50"])].index,
    axis=0,
)

for year in df["Year"].unique():
    ydf = df[df["Year"] == year]
    for _, gdf in ydf.groupby("Education"):
        # Trying to estimate the number of children born. Don't have multi-decade data, so assuming
        # the "Total Children" difference between 40-50 group and 30-39 group approximates the number
        # of children born to 40-50 year old mothers
        gdf = gdf.sort_values("Age Group", ascending=True)
        births_up_to = gdf["Total Children"].shift(1, fill_value=0)
        df.loc[gdf.index, "Total Births"] = gdf["Total Children"] - births_up_to
df.head()

,Year,Education,Age Group,Marital Status,Children Ever Born per Woman,Total Women,Childlessness,Total Children,Total Births
18,2012,Associate's degree,20 - 29,All Marital Statuses,0.734,2007000.0,0.584,1473138.0,1473138.0
19,2012,Bachelor's degree,20 - 29,All Marital Statuses,0.294,4397000.0,0.794,1292718.0,1292718.0
20,2012,Graduate or professional degree,20 - 29,All Marital Statuses,0.310,1041000.0,0.801,322710.0,322710.0
21,2012,High school graduate,20 - 29,All Marital Statuses,0.955,5305000.0,0.454,5066275.0,5066275.0
22,2012,Not a high school graduate,20 - 29,All Marital Statuses,1.482,2008000.0,0.275,2975856.0,2975856.0


In [4]:
df["Birth Rate"] = df["Total Births"] / df["Total Women"]
df.head()

,Year,Education,Age Group,Marital Status,Children Ever Born per Woman,Total Women,Childlessness,Total Children,Total Births,Birth Rate
18,2012,Associate's degree,20 - 29,All Marital Statuses,0.734,2007000.0,0.584,1473138.0,1473138.0,0.734
19,2012,Bachelor's degree,20 - 29,All Marital Statuses,0.294,4397000.0,0.794,1292718.0,1292718.0,0.294
20,2012,Graduate or professional degree,20 - 29,All Marital Statuses,0.310,1041000.0,0.801,322710.0,322710.0,0.310
21,2012,High school graduate,20 - 29,All Marital Statuses,0.955,5305000.0,0.454,5066275.0,5066275.0,0.955
22,2012,Not a high school graduate,20 - 29,All Marital Statuses,1.482,2008000.0,0.275,2975856.0,2975856.0,1.482


In [5]:
df.describe()

,Year,Children Ever Born per Woman,Total Women,Childlessness,Total Children,Total Births,Birth Rate
count,90.000000,90.000000,9.000000e+01,90.000000,9.000000e+01,9.000000e+01,90.000000
mean,2016.000000,1.489278,3.645600e+06,0.342900,5.208723e+06,2.530541e+06,0.728516
std,2.844273,0.720141,1.607000e+06,0.237758,3.005329e+06,1.472514e+06,0.375945
min,2012.000000,0.252000,1.041000e+06,0.077000,3.227100e+05,1.639400e+04,0.005147
25%,2014.000000,0.860000,2.233750e+06,0.150250,3.280426e+06,1.296234e+06,0.422102
50%,2016.000000,1.713500,3.512000e+06,0.227000,5.056691e+06,2.541895e+06,0.649224
75%,2018.000000,1.999750,5.176750e+06,0.503500,6.779255e+06,3.367580e+06,1.052632
max,2020.000000,2.710000,6.601000e+06,0.840000,1.336900e+07,6.596352e+06,1.671732


# Population Trends

In [6]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.colors import colorbrewer
from plotly.subplots import make_subplots

In [7]:
# Plot the child and female populations over the decade

fig = make_subplots(specs=[[{"secondary_y": True}]])

by_year = df.groupby("Year", as_index=False).sum()

by_year["Children per Woman"] = by_year["Total Children"] / by_year["Total Women"]
# Calculate YoY Change in Children
by_year["Children Change"] = by_year["Total Children"] - by_year["Total Children"].shift()
by_year["Children Percent Change"] = by_year["Children Change"] / by_year["Total Children"].shift() * 100
# Calculate YoY Change in Women
by_year["Women Change"] = by_year["Total Women"] - by_year["Total Women"].shift()
by_year["Women Percent Change"] = by_year["Women Change"] / by_year["Total Women"].shift() * 100

CHILDREN_COLOR = colorbrewer.Dark2_r[0]
WOMEN_COLOR = colorbrewer.Dark2_r[1]

# Add children traces
fig.add_trace(
        go.Scatter(
                x=by_year["Year"],
                y=by_year["Total Children"],
                name="Children",
                line=dict(color=CHILDREN_COLOR),
                legendgroup="Total",
                legendgrouptitle_text="Total",
        )
)
fig.add_trace(
        go.Scatter(
                x=by_year["Year"],
                y=by_year["Children Change"],
                name="Children",
                line=dict(color=CHILDREN_COLOR, dash="dot"),
                legendgroup="YoY Change",
                legendgrouptitle_text="YoY Change",
                hovertext=by_year["Children Percent Change"].round(1).astype(str) + "%",
        ),
        secondary_y=True
)
# Add women traces
fig.add_trace(
        go.Scatter(
                x=by_year["Year"],
                y=by_year["Total Women"],
                name="Women",
                line=dict(color=WOMEN_COLOR),
                legendgroup="Total",
        )
)
fig.add_trace(
        go.Scatter(
                x=by_year["Year"],
                y=by_year["Women Change"],
                name="Women",
                line=dict(color=WOMEN_COLOR, dash="dot"),
                legendgroup="YoY Change",
                hovertext=by_year["Women Percent Change"].round(1).astype(str) + "%",
        ),
        secondary_y=True,
)

fig.update_layout(
        title="Women Aged 20-50 and Their Children by Year",
        template="plotly_dark",
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)'
)
fig.update_yaxes(title_text="Total", secondary_y=False)
fig.update_yaxes(title_text="YoY Change", secondary_y=True)


fig

**Commentary**

The change in the population of children from 2018 to 2020 is fairly radical. There is a regular trend towards fewer and fewer children, but from 2018 to 2020 the trend accelerates. This is not a product of COVID, as the effects of US COVID policy on births would not be felt any earlier than December 2020. This sharp decline is suggestive that a segment (or segments) of the female population had significantly fewer births compared to the women of that segment 2 years prior. The norm has been between 1-2% decreases in the number of children YoY – 2020 sees a 5% decrease.

Notably, 2020 is the first year where this population of women *decreases*. This would require a smaller group of women to have more children to offset any population loss.

# Fertility Trends by Education Segments

In [8]:
def make_education_filtered_df(df):
    # Keeping things simple and only looking at the 3 most common education levels
    education_level_map = {
            "High school graduate": "High School",
            "Some college, no degree": "High School",
            # "Associate's degree": "High School", # I apologize to all the associates degree holders
            "Bachelor's degree": "Bachelor's",
            "Graduate or professional degree": "Graduate"
    }
    df["Education"] = df["Education"].replace(education_level_map)
    df = df.filter(
        df[df["Education"].isin(education_level_map.values())].index,
        axis=0,
    )
    df = df.groupby(["Year", "Education", "Age Group"], as_index=False).sum()
    df["Children Ever Born per Woman"] = df["Total Children"] / df["Total Women"]
    df["Birth Rate"] = df["Total Births"] / df["Total Women"]
    df = df.drop(columns=["Childlessness"])
    return df


In [9]:
# The change in children of 20-50 year old women broken down by educational attainment

fig = make_subplots(
        rows=3,
        specs=[[{"secondary_y": True}], [{"secondary_y": True}], [{}]],
        subplot_titles=("Children", "Women", "Fertility Trends by Education Segment <br><sup>Difference Between Children's and Womens' YoY Change Rates</sup>"),
        shared_xaxes=True
)

fdf = make_education_filtered_df(df.copy())

education_rank_order = ["High School", "Bachelor's", "Graduate"]

for i, (education, edf) in enumerate(fdf.groupby("Education")):
        by_year = edf.groupby("Year", as_index=False).sum()

        by_year["Children per Woman"] = by_year["Total Children"] / by_year["Total Women"]
        # Calculate YoY Change in Children
        by_year["Children Change"] = by_year["Total Children"] - by_year["Total Children"].shift()
        by_year["Children Percent Change"] = by_year["Children Change"] / by_year["Total Children"].shift() * 100
        # Calculate YoY Change in Women
        by_year["Women Change"] = by_year["Total Women"] - by_year["Total Women"].shift()
        by_year["Women Percent Change"] = by_year["Women Change"] / by_year["Total Women"].shift() * 100

        education_color = colorbrewer.Dark2[i]

        # Add children traces
        fig.add_trace(
                go.Scatter(
                        x=by_year["Year"],
                        y=by_year["Total Children"],
                        name="Total Children",
                        line=dict(color=education_color),
                        legendgroup=education,
                        legendgrouptitle_text=education,
                        legendrank=education_rank_order.index(education),
                        hovertext=education
                ),
                row=1,
                col=1,
        )
        fig.add_trace(
                go.Scatter(
                        x=by_year["Year"],
                        y=by_year["Children Change"],
                        name="Children YoY Change",
                        line=dict(color=education_color, dash="dot"),
                        legendgroup=education,
                        hovertext=education + " " + by_year["Children Percent Change"].round(1).astype(str) + "%",
                ),
                row=1,
                col=1,
                secondary_y=True,
        )
        # Add women traces
        fig.add_trace(
                go.Scatter(
                        x=by_year["Year"],
                        y=by_year["Total Women"],
                        name="Total Women",
                        line=dict(color=education_color),
                        legendgroup=education,
                        hovertext=education
                ),
                row=2,
                col=1,
        )
        fig.add_trace(
                go.Scatter(
                        x=by_year["Year"],
                        y=by_year["Women Change"],
                        name="Women YoY Change",
                        line=dict(color=education_color, dash="dot"),
                        legendgroup=education,
                        hovertext=education + " " + by_year["Women Percent Change"].round(1).astype(str) + "%",
                ),
                row=2,
                col=1,
                secondary_y=True,
        )

        # Point differential traces
        fig.add_trace(
                go.Scatter(
                        x=by_year["Year"],
                        y=(by_year["Children Percent Change"] - by_year["Women Percent Change"]).round(1),
                        name="% Points",
                        line=dict(color=education_color, dash="longdash"),
                        legendgroup=education,
                ),
                row=3,
                col=1,
        )

fig.update_layout(
        title="Women Aged 20-50 and Their Children by Year and Education Level",
        template="plotly_dark",
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)'
)
fig.update_yaxes(title_text="Total Children", row=1, secondary_y=False)
fig.update_yaxes(title_text="Total Women", row=2, secondary_y=False)
fig.update_yaxes(title_text="% Points", row=3, secondary_y=False)
fig.update_yaxes(title_text="YoY Change", secondary_y=True)

fig.update_layout(height=600, hovermode="x")
fig

**Commentary**

Women with graduate degrees are trending better than those who stopped at bachelor's degrees or high school diplomas. Lately, this segment has seen more growth in the population of their children than growth in the population of women, suggesting improved conditions for fertility. The other two groups are in a chronic deficit. They have fewer children than the change in their population size would expect them to experience.

It's important to use the point differential over total populations or YoY change. The point differential accounts for the fact that these education segments are on the move, while the number or change in children born to women who attained bachelor's degrees would mislead you to believe that segment is doing better than it did in the past.

# Advancing the Statistic

While point differential is better than raw change, point differential doesn't account for the fact that adding 1 woman to the population ought to increase the number of children by the crude fertility rate. If the crude fertility rate of a population is 1.7, then the expected change in children after adding 1 women is 1.7. Therefore, a better method for assessing the segment's trend is one that accounts for expectations.

A raw figure would be the difference in expected children and observed children from period $t_0$ to $t_1$.

**Variables**

$C$: Children

$W$: Women

$r$: Children per woman (child rate)

$t_0$: Beginning of period

$t_1$: End of period

$$\hat{C} = W_{t_1} r_{t_0}$$
$$e = C_{t_1} - \hat{C}$$

When the segment has a stable child rate, the error should approximate 0.

To standardize this we can divide by the observed and predicted children. This only really holds as long as observed or predicted children aren't 0, but we have bigger problems if we're running into that edge case.

$$\sigma = \frac{e}{C_{t_1} \hat{C}}$$

$$=\frac{C_{t_1} - \hat{C}}{C_{t_1} \hat{C}}$$

$$=\frac{1}{\hat{C}} - \frac{1}{C_{t_1}}$$


If the expectation is 1 child, but the observed is 2, $\sigma=0.5$. If the expectation is 2 children but the observed is 1, $\sigma=-0.5$. If the expectation is 100 children but the observed is 1, $\sigma=-.99$.

**Notes**

$-1 < \sigma < 1$ is easily interpretable, but I found interpreting $\sigma$ plots with large populations difficult as $\sigma$ becomes impossibly small. I convert population totals to millions for legibility in plots. The constant has a linear impact on $\sigma$.

$C=xc$

$W=xw$

$\hat{C} = xw_{t_1} r_{t_0}$

$$\sigma=\frac{1}{\hat{C}} - \frac{1}{C_{t_1}}$$

$$=\frac{1}{xw_{t_1} r_{t_0}} - \frac{1}{xc_{t_1}}$$

$$=\frac{1}{x} \cdot (\frac{1}{w_{t_1} r_{t_0}} - \frac{1}{c_{t_1}})$$

In [10]:
def sigma(r: float, w: int, c: int) -> float:
    """Calculate the standardized error between the predicted number of children and the observed number of children.

    :param r:   The child rate to predict with.
    :param w:   The observed number of women.
    :param c:   The observed number of children.
    :return:    The standardized error between the predicted number of children and the observed number of children.
    """
    # Calculate the error between the predicted number of children and the actual number of children
    predicted = w * r
    error = c - predicted
    # Return the standardized error
    return error / (c * predicted)

fdf = make_education_filtered_df(df.copy())
# Convert to millions
fdf["Women (Millions)"] = fdf["Total Women"] / 1_000_000
fdf["Children (Millions)"] = fdf["Total Children"] / 1_000_000

ed_by_year_dataframes = []
for education, edf in fdf.groupby("Education"):
    by_year = edf.groupby("Year", as_index=False).agg(
        {
            "Children (Millions)": "sum",
            "Women (Millions)": "sum",
        }
    )
    by_year["Child Rate"] = by_year["Children (Millions)"] / by_year["Women (Millions)"]
    by_year["Sigma"] = sigma(by_year["Child Rate"].shift(), by_year["Women (Millions)"], by_year["Children (Millions)"])
    by_year["Education"] = education
    ed_by_year_dataframes.append(by_year)

fdf = pd.concat(ed_by_year_dataframes)
# Create a normalized sigma column for a second y-axis
fdf["Normalized Sigma"] = fdf["Sigma"] / fdf["Sigma"].abs().max()

fig = px.scatter(
    fdf,
    x="Year",
    y="Sigma",
    color="Education",
    size="Women (Millions)",
    title="Fertility Trends by Education Segment<br><sup>YoY Sigma</sup>",
    template="plotly_dark",
    hover_name="Education",
    hover_data={
        "Year": True,
        "Children (Millions)": ":.2f",
        "Women (Millions)": ":.2f",
    },
    labels={
        "Sigma": "Sigma (YoY)",
    },
    color_discrete_sequence=colorbrewer.Dark2,           
)
# Update legend rank
fig.for_each_trace(
    lambda trace: trace.update(
        legendrank=education_rank_order.index(trace.name)
    )
)

fig.update_layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    hovermode="x",
)
fig.update_traces(mode="markers+lines", marker=dict(symbol="arrow"))
fig

**Commentary**

Using sigma instead of point differentials reveals a commonality: the fertility of highly educated women is increasing. The story for the other two is similar, with a notable change. Both are slightly decreasing as observed before, but now fertility for women with bachelor's degrees is attriting slightly more relative to those with high school diplomas.

# The Paradox


In [11]:
# Plot the birth rate for each age group and education segment

gdf = df.groupby(["Year", "Education"], as_index=False).agg({"Birth Rate": "sum", "Children Ever Born per Woman": "sum", "Total Women": "sum", "Total Children": "sum", "Total Births": "sum"})
gdf["Age Group"] = "20 - 50"
gdf = pd.concat([gdf, df])


for age_group in ["20 - 29", "30 - 39", "40 - 50", "20 - 50"]:
    fdf: pd.DataFrame = make_education_filtered_df(gdf[gdf["Age Group"] == age_group].copy())
    # Convert to millions
    fdf["Women (Millions)"] = fdf["Total Women"] / 1_000_000
    fdf["Children (Millions)"] = fdf["Total Children"] / 1_000_000

    fig = px.scatter(
        fdf,
        x="Year",
        y="Children Ever Born per Woman",
        color="Education",
        size="Women (Millions)",
        title=f"Fertility Trends for Women Aged {age_group} by Education Segment<br><sup>Children Ever Born to Segment</sup>",
        template="plotly_dark",
        hover_name="Education",
        hover_data={
            "Year": True,
            "Children (Millions)": ":.2f",
            "Women (Millions)": ":.2f",
        },
        color_discrete_sequence=colorbrewer.Dark2,           
    )
    # Update legend rank
    fig.for_each_trace(
        lambda trace: trace.update(
            legendrank=education_rank_order.index(trace.name)
        )
    )

    fig.update_layout(
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        hovermode="x",
    )
    fig.update_traces(mode="markers+lines", marker=dict(symbol="arrow"))
    fig.show()
gdf

,Year,Education,Birth Rate,Children Ever Born per Woman,Total Women,Total Children,Total Births,Age Group,Marital Status,Childlessness
0,2012,Associate's degree,2.330833,4.401,7214000.0,11066905.0,5484824.0,20 - 50,NaN,NaN
1,2012,Bachelor's degree,2.005512,3.511,14413000.0,17540611.0,9669385.0,20 - 50,NaN,NaN
2,2012,Graduate or professional degree,1.871993,3.256,6680000.0,8599977.0,4764450.0,20 - 50,NaN,NaN
3,2012,High school graduate,2.519022,5.071,16260000.0,27642657.0,13369004.0,20 - 50,NaN,NaN
4,2012,Not a high school graduate,2.900503,6.718,6294000.0,14197288.0,5915597.0,20 - 50,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
175,2020,Bachelor's degree,0.482532,1.863,5899000.0,10989837.0,2846457.0,40 - 50,All Marital Statuses,0.190
176,2020,Graduate or professional degree,0.492379,1.702,4196000.0,7141592.0,2066021.0,40 - 50,All Marital Statuses,0.192
177,2020,High school graduate,0.546985,2.092,4869000.0,10185948.0,2663268.0,40 - 50,All Marital Statuses,0.134
178,2020,Not a high school graduate,0.652835,2.676,1810000.0,4843560.0,1181631.0,40 - 50,All Marital Statuses,0.077


**Commentary**

The above plots display an enigma: graduate-educated women aged 20-50 have more children per woman than those with bachelor's degrees, but they have fewer children per woman than those with bachelor's degrees in age groups 20-29, 30-39, and 40-50\*.

'Children ever born' is a metric that gets closer to the age cohort's true contribution to the next generation over time. The 40-50 cohort is the most stable metric while a 15-19 cohort would be the least stable.

\* There is that small lead in 20-29 in 2012, 2014, and 2020, but the graduate population is so small for the age group that it shouldn't be driving such a stark change for even those years (See [Appendix, 1](#1-total-women-by-age-group-and-education-segment))

| Institude for Family Studies | Statista | CDC |
| :---: | :---: | :---: |
| ![](https://ifstudies.org/ifs-admin/resources/figure-1edfertility-copy-2-w640.png) | ![](https://www.statista.com/graphic/1/1238575/total-fertility-rate-us-education.jpg) | [National Vital Statistics Reports Volume 70 Number 5 May 12, 2021 (Page 3)](https://www.cdc.gov/nchs/data/nvsr/nvsr70/nvsr70-05-508.pdf) |


## Simpson's Paradox

The inversion of the graduate and bachelor's rates is an example of a statistical phenomenon colloquially known as "Simpson's Paradox". It's a where a trend in subgroups disappears or reverses when combined.

| Interpretation | Correlation | Vectors |
| :-- | :---: | :---: |
| **Plot** | ![](https://upload.wikimedia.org/wikipedia/commons/thumb/f/fb/Simpsons_paradox_-_animation.gif/440px-Simpsons_paradox_-_animation.gif) | ![](https://upload.wikimedia.org/wikipedia/commons/thumb/c/cd/Simpson_paradox_vectors.svg/440px-Simpson_paradox_vectors.svg.png) |
| **Rationale** | | Each of the blue vectors have stronger slopes than their categorical counterparts in the orange. <br> While the slopes of $B_1$ > $L_1$ and $B_2$ > $L_2$, $L_2$ > $B_1$ and its magnitude combined with the<br>difference in angles is substantial enough to outweigh the marginal leads of each $B_i$ over $L_i$. |

**Extras**

- [Wikipedia](https://en.wikipedia.org/wiki/Simpson's_paradox)
- [Stanford](https://plato.stanford.edu/entries/paradox-simpson/#:~:text=Simpson's%20Paradox%20is%20a%20statistical,population%20is%20divided%20into%20subpopulations.)


In [12]:
gdf = make_education_filtered_df(df.copy()).groupby(["Year", "Education"], as_index=False).agg({"Birth Rate": "sum", "Children Ever Born per Woman": "sum", "Total Women": "sum", "Total Children": "sum", "Total Births": "sum"})
gdf["Age Group"] = "20 - 50"
gdf = pd.concat([gdf, df])
gdf = make_education_filtered_df(gdf)
gdf = gdf[gdf["Year"] == 2020]

gdf["Total Women (Millions)"] = (gdf["Total Women"] / 1000000).round(1)
gdf["Total Children (Millions)"] = (gdf["Total Children"] / 1000000).round(1)
gdf["Children Ever Born per Woman"] = gdf["Children Ever Born per Woman"].round(1)

age_group_order = ["20 - 29", "30 - 39", "40 - 50", "20 - 50"]
metric_order = ["Total Children (Millions)", "Total Women (Millions)", "Children Ever Born per Woman"]
piv = gdf.pivot_table(index="Education", columns="Age Group", values=metric_order)

# Reorder so that the age groups of each metric look like they are
# culminating in the 20 - 50 age group
i = 0
for metric in metric_order:
    for age_group in age_group_order:
        piv.insert(i, (metric, age_group), piv.pop((metric, age_group)))
        i += 1

piv

Total Children (Millions)                           
Age Group                     20 - 29 30 - 39 40 - 50 20 - 50   
Education                                                       
Bachelor's                        1.5     8.1    11.0    20.7  \
Graduate                          0.4     5.1     7.1    12.6   
High School                       6.0    13.0    16.6    35.7   

            Total Women (Millions)                           
Age Group                  20 - 29 30 - 39 40 - 50 20 - 50   
Education                                                    
Bachelor's                     6.1     6.5     5.9    18.5  \
Graduate                       1.5     4.3     4.2    10.0   
High School                   10.9     7.4     7.9    26.2   

            Children Ever Born per Woman                          
Age Group                        20 - 29 30 - 39 40 - 50 20 - 50  
Education                                                         
Bachelor's                           0.3     1.3     1.9     1.1  
Graduate                             0.3     1.2     1.7     1.3  
High School                          0.5     1.8     2.1     1.4

In [13]:
size = {
    "20 - 29": 10,
    "30 - 39": 20,
    "40 - 50": 30,
    "20 - 50": 40,
}
fdf = make_education_filtered_df(df.copy()).groupby(["Year", "Education"], as_index=False).agg({"Birth Rate": "sum", "Children Ever Born per Woman": "sum", "Total Women": "sum", "Total Children": "sum", "Total Births": "sum"})
fdf["Age Group"] = "20 - 50"
fdf = pd.concat([fdf, df])
fdf = make_education_filtered_df(fdf)
fdf["Size"] = fdf["Age Group"].replace(size)
fdf["Education Order"] = fdf["Education"].apply(lambda x: education_rank_order.index(x))
fdf = fdf.sort_values(["Education Order", "Size"])


YEAR = 2020
fdf = fdf[fdf["Year"] == YEAR]


def draw_line(fig, df, education, age_group, color):
    row = df[(df["Education"] == education) & (df["Age Group"] == age_group)].iloc[0]
    fig.add_shape(
        type="line",
        x0=0,
        y0=0,
        x1=row["Total Women"],
        y1=row["Total Children"],
        layer="below",
        line=dict(
            color=color,
            width=2,
            dash="dot",
        ),
    )

for age_group in ["20 - 29", "30 - 39", "40 - 50", "20 - 50"]:

    fig = px.scatter(
        fdf,
        x="Total Women",
        y="Total Children",
        symbol="Age Group",
        color="Education",
        size="Size",
        template="plotly_dark",
        color_discrete_sequence=[colorbrewer.Dark2[2], colorbrewer.Dark2[0], colorbrewer.Dark2[1]],
        title=f"Children per Woman Aged {age_group} Slopes by Education Segment ({YEAR})",
        hover_data=["Children Ever Born per Woman"]
    )

    education_groups = fdf["Education"].unique()
    for education, age_group, color in zip(education_groups, [age_group] * len(education), [colorbrewer.Dark2[2], colorbrewer.Dark2[0], colorbrewer.Dark2[1]]):
        draw_line(
            fig,
            fdf,
            education,
            age_group,
            color
        )

    fig.update_layout(
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
    )
    fig.show()

**commentary**

The results of this again demonstrate Simpson's Paradox. However, the cause of it is now observable.

For each age group we see that the Children Ever Born per Woman is about the same between women with bachelor's and graduate degrees. The bachelor's tend to have very slightly more children – not as many more as one would expect, but I'll be covering that later. The real differences are seen in the total number of women.

The number of women who attain high school or bachelor's are fairly fixed after the ages of 18 and 22. However, people attain MDs, PhDs, MBAs, and JDs all through their late 20s thru 30s. More than half of the 20 - 29 bachelor's attainers are immediately thrown out of the 20-29 graduate group because they just aren't old enough to have attained the degree (even though they will later). Maybe in 10 years they will be classified and counted together...

This is where the paradox of your mother's education comes into play. Speaking for myself and the Berkeley admissions department here, I frequently looked at the child rates for each group and expected a simple average to be roughly what made the 20-50 group. It was close to true for the high school attainers, and close to true for the bachelor's attainers, but very much not true for the graduate attainers. This is thanks to the enormous discrepancy between 20 - 29 and 30 - 39 graduates. Because the influence of each age subgroups on the 20-50 group is proportional to the size of the subgroup. All that had to be done was changing the mental math from this

$$\frac{
    \frac{
    \text{20 to 29 children}
    }{\text{20 to 29 women}}
+ \frac{
    \text{30 to 39 children}
    }{\text{30 to 39 women}}
+ \frac{
    \text{40 to 49 children}}{\text{40 to 49 women}}
}{3}$$

to this

$$
\frac{
    \text{20 to 29 children} + \text{30 to 39 children} + \text{40 to 49 children}
}{
    \text{20 to 29 women} + \text{30 to 39 women} + \text{40 to 49 women}
}
$$

or to old faithful

$$\sum{xp(x)}$$

The solution to a problem that nearly turned blue hair red at Berkeley was to remember the definition of an average.

## The Effect on $\sigma$

In [14]:

gdf = df.groupby(["Year", "Education"], as_index=False).agg({"Birth Rate": "sum", "Children Ever Born per Woman": "sum", "Total Women": "sum", "Total Children": "sum", "Total Births": "sum"})
gdf["Age Group"] = "20 - 50"
gdf = pd.concat([gdf, df], ignore_index=True)
gdf = make_education_filtered_df(gdf)

# Convert to millions
gdf["Women (Millions)"] = gdf["Total Women"] / 1_000_000
gdf["Children (Millions)"] = gdf["Total Children"] / 1_000_000

for age_group in ["20 - 29", "30 - 39", "40 - 50", "20 - 50"]:
    fdf = gdf[gdf["Age Group"] == age_group]
    ed_by_year_dataframes = []
    for education, edf in fdf.groupby("Education"):
        by_year = edf.groupby("Year", as_index=False).agg(
            {
                "Children (Millions)": "sum",
                "Women (Millions)": "sum",
            }
        )
        by_year["Child Rate"] = by_year["Children (Millions)"] / by_year["Women (Millions)"]
        by_year["Sigma"] = sigma(by_year["Child Rate"].shift(), by_year["Women (Millions)"], by_year["Children (Millions)"])
        by_year["Education"] = education
        ed_by_year_dataframes.append(by_year)

    fdf = pd.concat(ed_by_year_dataframes)
    # Create a normalized sigma column for a second y-axis
    fdf["Normalized Sigma"] = fdf["Sigma"] / fdf["Sigma"].abs().max()

    fig = px.scatter(
        fdf,
        x="Year",
        y="Sigma",
        color="Education",
        size="Women (Millions)",
        title=f"Fertility Trends for Women Aged {age_group} by Education Segment<br><sup>YoY Sigma</sup>",
        template="plotly_dark",
        hover_name="Education",
        hover_data={
            "Year": True,
            "Children (Millions)": ":.2f",
            "Women (Millions)": ":.2f",
        },
        labels={
            "Sigma": "Sigma (YoY)",
        },
        color_discrete_sequence=colorbrewer.Dark2,           
    )
    # Update legend rank
    fig.for_each_trace(
        lambda trace: trace.update(
            legendrank=education_rank_order.index(trace.name)
        )
    )

    fig.update_layout(
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        hovermode="x",
    )
    fig.update_traces(mode="markers+lines", marker=dict(symbol="arrow"))
    fig.show()

# Appendix

Additional plots for reference.

## 1) Total Women by Age Group and Education Segment

In [15]:
fdf = make_education_filtered_df(df)
# Aggregate by age group and education segment and calculate the standard deviation
fdf = fdf.groupby(["Age Group", "Education"], as_index=False).agg({"Total Women": ["sum", "std"]})
# Flatten hierarchical column index
fdf = fdf.set_axis([' '.join(col).strip() for col in fdf.columns], axis=1)

fdf["Education Order"] = fdf["Education"].map(lambda x: education_rank_order.index(x))
fdf = fdf.sort_values(["Age Group", "Education Order"])

fig = px.bar(
    fdf,
    x="Age Group",
    y="Total Women sum",
    color="Education",
    title="Total Women by Age Group and Education Segment",
    template="plotly_dark",
    barmode="group",
    # Trying to keep colors consistent across plots. Plots are generated by education label alphabetically, so H is last, B first, G second
    color_discrete_sequence=[colorbrewer.Dark2[2], colorbrewer.Dark2[0], colorbrewer.Dark2[1]],
    error_y="Total Women std",
    labels={
        "Total Women sum": "Total Women"
    }
)
fig.update_layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
)
fig
